In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.7393,-90.3518,72.86,85,9,8.23,EC,2022-04-28 23:38:37
1,1,Rikitea,-23.1203,-134.9692,77.38,64,21,18.39,PF,2022-04-28 23:33:08
2,2,Puerto Madryn,-42.7692,-65.0385,47.05,33,32,6.85,AR,2022-04-28 23:41:54
3,3,Caravelas,-17.7125,-39.2481,75.31,85,0,5.35,BR,2022-04-28 23:40:08
4,4,Punta Arenas,-53.1500,-70.9167,42.91,81,40,21.85,CL,2022-04-28 23:41:55


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=8)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=8)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Rikitea,-23.1203,-134.9692,77.38,64,21,18.39,PF,2022-04-28 23:33:08
3,3,Caravelas,-17.7125,-39.2481,75.31,85,0,5.35,BR,2022-04-28 23:40:08
14,14,Aripuana,-9.1667,-60.6333,77.22,77,100,0.27,BR,2022-04-28 23:41:03
15,15,Mount Isa,-20.7333,139.5000,82.17,57,29,12.66,AU,2022-04-28 23:42:01
23,23,Avarua,-21.2078,-159.7750,82.45,74,20,17.27,CK,2022-04-28 23:33:13
29,29,Hithadhoo,-0.6000,73.0833,84.83,69,76,7.38,MV,2022-04-28 23:38:33
35,35,Adrar,20.5022,-10.0711,75.45,12,0,8.30,MR,2022-04-28 23:42:10
36,36,Kerema,-7.9631,145.7785,83.28,72,99,3.83,PG,2022-04-28 23:42:11
44,44,Mae Sai,20.4335,99.8762,79.83,53,25,0.60,TH,2022-04-28 23:42:14
47,47,Butaritari,3.0707,172.7902,81.97,79,99,16.17,KI,2022-04-28 23:43:16


In [12]:
preferred_cities_df.count()

City_ID       40
City          40
Lat           40
Lng           40
Max Temp      40
Humidity      40
Cloudiness    40
Wind Speed    40
Country       40
Date          40
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,77.38,-23.1203,-134.9692,
3,Caravelas,BR,75.31,-17.7125,-39.2481,
14,Aripuana,BR,77.22,-9.1667,-60.6333,
15,Mount Isa,AU,82.17,-20.7333,139.5000,
23,Avarua,CK,82.45,-21.2078,-159.7750,
29,Hithadhoo,MV,84.83,-0.6000,73.0833,
35,Adrar,MR,75.45,20.5022,-10.0711,
36,Kerema,PG,83.28,-7.9631,145.7785,
44,Mae Sai,TH,79.83,20.4335,99.8762,
47,Butaritari,KI,81.97,3.0707,172.7902,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,77.38,-23.1203,-134.9692,People ThankYou
3,Caravelas,BR,75.31,-17.7125,-39.2481,Pousada dos Navegantes
14,Aripuana,BR,77.22,-9.1667,-60.6333,
15,Mount Isa,AU,82.17,-20.7333,139.5000,ibis Styles Mt Isa Verona
23,Avarua,CK,82.45,-21.2078,-159.7750,Paradise Inn
29,Hithadhoo,MV,84.83,-0.6000,73.0833,Scoop Guest House
35,Adrar,MR,75.45,20.5022,-10.0711,
36,Kerema,PG,83.28,-7.9631,145.7785,Frena Lodge
44,Mae Sai,TH,79.83,20.4335,99.8762,Piyaporn Hill Paradise Hotel
47,Butaritari,KI,81.97,3.0707,172.7902,Isles Sunset Lodge


In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [21]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))